In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark
!pip install graphframes
!apt-get install scala
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install networkx

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 17.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=a2b6c5fbb72835e5bc07549113dc823c03deae7ee2f98ce36f50c43377e63325
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 154 kB 21.0 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libhawtjni-runtime-java libjansi-java libjansi-native-java libjline2-java
  scala-library scala-parser-combinators scala-xml
Suggested packages:
  scala-doc
The following NEW packages will be installed:
  libhawtjni-runtime-java libjansi-java libjansi-native-java libjline2-java
  scala scala-library scala-parser-combinators scala-xml
0 upgraded, 8 newly installed, 0 to remove an

In [3]:
import os
import sys
import pandas as pd
import numpy as np
import networkx as nx
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os
import math
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType 
from graphframes import *

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
sc = SparkContext().getOrCreate()

In [6]:
count_df = pd.read_csv('/content/drive/MyDrive/Big Data/data_summary/tweet_retweet_counts.csv')
high_count_user = count_df[count_df['tweet_retweet_count'] >= 5]['user_id'].values

In [7]:
df_follower = pd.read_csv('/content/drive/MyDrive/Big Data/data_summary/all_followers.csv')
df_follower.dropna(inplace = True)
df_follower.drop_duplicates(inplace = True)
df_follower.drop(columns=['Unnamed: 0'], inplace=True)

In [8]:
df_follower['followers'] = df_follower['followers'].astype(int)
df_follower['user_id'] = df_follower['user_id'].astype(int)
df_follower.info()
df_follower.groupby(['user_id']).count().sort_values(['followers'], ascending=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3683243 entries, 0 to 3683242
Data columns (total 2 columns):
 #   Column     Dtype
---  ------     -----
 0   followers  int64
 1   user_id    int64
dtypes: int64(2)
memory usage: 84.3 MB


,followers
user_id,
961387417759567872,5000
715349560491520000,5000
161769014,5000
158253154,5000
152007388,5000
...,...
834246099858759681,1
743041000076156928,1
743258531378278400,1


In [62]:
df_follower_count = df_follower.groupby(['user_id']).count().sort_values(['followers'], ascending=False)
high_follower = list(df_follower_count[df_follower_count['followers'] > 2679].index)

In [63]:
df_follower_count.describe()

,followers
count,2267.000000
mean,1624.721217
std,1767.487664
min,1.000000
25%,179.500000
50%,813.000000
75%,2679.500000
max,5000.000000


In [64]:
get_col_count_df_grouped = count_df[count_df['user_id'].isin(high_follower)].groupby('user_id')
user_id = list(get_col_count_df_grouped.mean().index)
percentage_fake = get_col_count_df_grouped.mean()['percentage_fake'].values
tweet_retweet_count = get_col_count_df_grouped.sum()['tweet_retweet_count'].values

In [65]:
new_dict_df = {'user_id' : user_id, 'percentage_fake': percentage_fake, 'tweet_retweet_count' : tweet_retweet_count}
filtered_df = pd.DataFrame.from_dict(new_dict_df)

In [66]:
def selected_rows(df, fake_percentage, tweet_retweet_count):
  df = df[df['percentage_fake'] >= fake_percentage]
  df = df[df['tweet_retweet_count'] >= tweet_retweet_count]
  print(len(df['user_id'].values))
  return df['user_id'].values

In [67]:
selected_user_id = selected_rows(filtered_df, 0.333333, 3)

71


In [101]:
def create_edge_df_nodes_df(df, selected_id_list):
  edge_df = df[df['user_id'].isin(selected_id_list)]
  edge_df.drop_duplicates(inplace = True)
  edge_df.rename(columns={'user_id': 'src', 'followers': 'dst'}, inplace = True)
  edge_df.reset_index(inplace = True)
  edge_df.drop(columns=['index'], inplace = True)

  selected_dst = list(edge_df['dst'].unique())
  selected_src = list(edge_df['src'].unique())

  for src in selected_src:
    if src not in selected_dst:
      selected_dst.append(src)

  nodes_dict = {'id' : selected_dst, 'node' : selected_dst}
  nodes_df = pd.DataFrame(nodes_dict)

  return (edge_df, nodes_df)

In [103]:
edge_df, nodes_df = create_edge_df_nodes_df(df_follower, selected_user_id)

In [105]:
edge_df.to_csv('edge_df.csv')
nodes_df.to_csv('nodes_df.csv')

In [100]:
selected_user_df

,dst,src
0,332696199,716905889
1,375835655,716905889
2,897630289282969600,716905889
3,796746831988453376,716905889
4,1015079685192146944,716905889
...,...,...
310774,940047133293535232,872230649310183425
310775,885873862646345728,872230649310183425
310776,709371556523335680,872230649310183425
310777,762878463711649792,872230649310183425


In [ ]:
nodes_df = pd.DataFrame.from_dict(nodes_dict)

In [25]:
count_df[count_df['user_id'].isin(high_follower)].groupby('user_id').mean().sort_values(by=['percentage_fake'], ascending=False).describe()

,tweet_real_news_count,tweet_fake_news_count,tweet_count,retweet_real_news_count,retweet_fake_news_count,retweet_count,tweet_retweet_fake_count,tweet_retweet_real_count,tweet_retweet_count,percentage_fake
count,761.000000,761.000000,761.000000,761.000000,761.000000,761.000000,761.000000,761.000000,761.000000,761.000000
mean,1.296978,1.289093,2.586071,0.386334,0.199737,0.586071,1.488830,1.683311,3.172142,0.456735
std,3.721361,5.203541,6.829369,1.012044,0.582237,1.325167,5.213868,3.874118,6.918553,0.446525
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.333333
75%,1.000000,1.000000,2.000000,0.000000,0.000000,1.000000,1.000000,2.000000,3.000000,1.000000
max,60.000000,102.000000,102.000000,15.000000,10.000000,20.000000,102.000000,60.000000,102.000000,1.000000


In [26]:
high_follower_fake_spreader = count_df[count_df['user_id'].isin(high_follower)].groupby('user_id').mean().sort_values(by=['percentage_fake'], ascending=False)
high_follower_fake_spreader[high_follower_fake_spreader['percentage_fake'] >= 0.333333]

,tweet_real_news_count,tweet_fake_news_count,tweet_count,retweet_real_news_count,retweet_fake_news_count,retweet_count,tweet_retweet_fake_count,tweet_retweet_real_count,tweet_retweet_count,percentage_fake
user_id,,,,,,,,,,
224491023,0.0,9.0,9.0,0.0,0.0,0.0,9.0,0.0,9.0,1.000000
4858879587,0.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,1.000000
4547869279,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.000000
301614418,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.000000
297561618,0.0,3.0,3.0,0.0,0.0,0.0,3.0,0.0,3.0,1.000000
...,...,...,...,...,...,...,...,...,...,...
1927978586,2.0,1.0,3.0,0.0,0.0,0.0,1.0,2.0,3.0,0.333333
385628927,0.0,1.0,1.0,4.0,1.0,5.0,2.0,4.0,6.0,0.333333
15266000,1.0,1.0,2.0,1.0,0.0,1.0,1.0,2.0,3.0,0.333333
